In [1]:
!pip3 install geopandas

In [2]:
import pandas as pd
import geopandas as gpd
import folium

# 관광지 데이터 로드 및 정렬
data = pd.read_csv('주요관광지점 입장객 2020_2022년.csv')
sorted_data = data.sort_values(by='총계', ascending=False)

# 데이터 분할
top_data = sorted_data.head(13)
next_data = sorted_data.iloc[13:21]
rest_data = sorted_data.iloc[21:]

# 제주도 읍면동 경계 데이터 로드
file_path = '제주도읍면동경계_final.gpkg'
jeju_boundary = gpd.read_file(file_path)
jeju_boundary_transformed = jeju_boundary.to_crs(epsg=4326)

# 제주도의 중심점 계산
jeju_center_transformed = jeju_boundary_transformed.geometry.unary_union.centroid

# Folium 지도 객체 생성
m_transformed = folium.Map(location=[jeju_center_transformed.y, jeju_center_transformed.x], zoom_start=10)

# 관광지 위치에 마커 추가
for lat, lon, name in zip(top_data['위도'], top_data['경도'], top_data['관광지']):
    folium.Marker([lat, lon], popup=name, icon=folium.Icon(color='red')).add_to(m_transformed)
for lat, lon, name in zip(next_data['위도'], next_data['경도'], next_data['관광지']):
    folium.Marker([lat, lon], popup=name, icon=folium.Icon(color='blue')).add_to(m_transformed)
for lat, lon, name in zip(rest_data['위도'], rest_data['경도'], rest_data['관광지']):
    folium.Marker([lat, lon], popup=name, icon=folium.Icon(color='green')).add_to(m_transformed)

# 제주도 경계 데이터 지도에 추가
folium.GeoJson(
    jeju_boundary_transformed.to_json(),
    style_function=lambda x: {'color': 'black', 'weight': 2, 'fill': False},
    name='Jeju Boundaries'
).add_to(m_transformed)

# 지도 표시
m_transformed


In [3]:
!pip3 install rtree
!pip3 install pygeos


In [12]:
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import HeatMap

# Load the boundary data
boundary_file_path = '제주도읍면동경계_final.gpkg'
jeju_boundary = gpd.read_file(boundary_file_path)

# Load the accommodation data
accom_data = pd.read_csv('Accom.csv')

# Filter out rows with NaN latitude or longitude values
accom_data = accom_data.dropna(subset=['latitude', 'longitude'])

# Convert latitude and longitude columns to geometry and set the CRS to EPSG:4326
accom_geometry = gpd.points_from_xy(accom_data['longitude'], accom_data['latitude'], crs='EPSG:4326')
accom_geo = gpd.GeoDataFrame(accom_data, geometry=accom_geometry)

# Perform spatial join to count the number of accommodations in each boundary
jeju_boundary_with_accom = gpd.sjoin(jeju_boundary, accom_geo, how='left', op='contains')
counts_per_boundary = jeju_boundary_with_accom.groupby('adm_cd')['title'].count()

# Add count data to the original boundary GeoDataFrame
jeju_boundary['Accom_Count'] = counts_per_boundary

# Create the map object
m = folium.Map(location=[33.376464, 126.531546], zoom_start=10)

# Create a heatmap layer based on the accommodation count
heat_data = jeju_boundary[['geometry', 'Accom_Count']].copy()
heat_data['Accom_Count'] = heat_data['Accom_Count'].fillna(0)  # Fill NaN values with 0 for the heatmap

# Convert MultiPolygon to individual Polygons and calculate centroids
heat_data['geometry'] = heat_data['geometry'].apply(lambda x: [g for g in x] if x.type == 'MultiPolygon' else [x])
heat_data = heat_data.explode('geometry')
heat_data['centroid'] = heat_data['geometry'].centroid

HeatMap(data=heat_data[['centroid', 'Accom_Count']], radius=15).add_to(m)

# Display the map
m


/Users/user/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py:3490: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/pf/26dznxks2c72mq_6mvqz7y6h0000gn/T/ipykernel_85424/2887136438.py:35: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  heat_data['geometry'] = heat_data['geometry'].apply(lambda x: [g for g in x] if x.type == 'MultiPolygon' else [x])


TypeError: 'MultiPolygon' object is not iterable

In [17]:
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import HeatMap

# Load the boundary data
boundary_file_path = '제주도읍면동경계_final.gpkg'
jeju_boundary = gpd.read_file(boundary_file_path)

# Load the accommodation data
accom_data = pd.read_csv('Accom.csv')

# Convert latitude and longitude columns to geometry and set the CRS to EPSG:4326
accom_geometry = gpd.points_from_xy(accom_data['longitude'], accom_data['latitude'], crs='EPSG:4326')
accom_geo = gpd.GeoDataFrame(accom_data, geometry=accom_geometry)

# Perform spatial join to count the number of accommodations in each boundary
jeju_boundary_with_accom = gpd.sjoin(jeju_boundary, accom_geo, how='left', op='contains')
counts_per_boundary = jeju_boundary_with_accom.groupby('adm_cd')['title'].count().reset_index()

# Merge counts_per_boundary with jeju_boundary
jeju_boundary = jeju_boundary.merge(counts_per_boundary, left_on='adm_cd', right_on='adm_cd', how='left')
jeju_boundary.rename(columns={'title': 'Accom_Count'}, inplace=True)
jeju_boundary['Accom_Count'] = jeju_boundary['Accom_Count'].fillna(0)

# Create the map object
m = folium.Map(location=[33.376464, 126.531546], zoom_start=10)

# Create a heatmap layer based on the accommodation count
heat_data = []
for _, row in jeju_boundary.iterrows():
    heat_data.append([row['geometry'].centroid.y, row['geometry'].centroid.x, row['Accom_Count']])

HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


/Users/user/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py:3490: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [23]:
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import HeatMap

# Load the boundary data
boundary_file_path = '제주도읍면동경계_final.gpkg'
jeju_boundary = gpd.read_file(boundary_file_path)

# Load the accommodation data
accom_data = pd.read_csv('Accom.csv')

# Convert latitude and longitude columns to geometry and set the CRS to EPSG:4326
accom_geometry = gpd.points_from_xy(accom_data['longitude'], accom_data['latitude'], crs='EPSG:4326')
accom_geo = gpd.GeoDataFrame(accom_data, geometry=accom_geometry)

# Create the map object
m = folium.Map(location=[33.376464, 126.531546], zoom_start=10)

# Iterate through each boundary and add a heatmap layer for the entire area of each boundary
for idx, row in jeju_boundary.iterrows():
    if row['geometry'].geom_type == 'Polygon':
        coords = row['geometry'].exterior.coords
        heat_data = [[coord[1], coord[0], 1] for coord in coords]  # Set count to 1 for the entire area
    elif row['geometry'].geom_type == 'MultiPolygon':
        heat_data = []
        for polygon in row['geometry']:
            coords = polygon.exterior.coords
            heat_data.extend([[coord[1], coord[0], 1] for coord in coords])  # Set count to 1 for the entire area
    
    HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


TypeError: 'MultiPolygon' object is not iterable